In [3]:
pip install openai

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


Initial Code with Extracting Sentiment/Reason + storage

In [43]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI

# Load environment variables
load_dotenv()

# Access the API key
api_key = os.getenv('OPENAI_API_KEY')

# Create an OpenAI client
client = OpenAI(api_key=api_key)

# Define the sentiment schema
sentiment_schema = {
    "type": "json_schema",
    "json_schema": {
        "name": "sentiment_response",
        "strict": True,
        "schema": {
            "type": "object",
            "properties": {
                "sentiment": {
                    "type": "boolean",
                    "description": "True if in favor of the proposition, false if against"
                },
                "reason": {
                    "type": "string",
                    "description": "Very brief core arguement in sentiment analysis"
                }
            },
            "required": ["sentiment", "reason"],
            "additionalProperties": False
        }
    }
}


def gpt4_call(user_prompt):
    try:
        # Load the schema from file
        system_message = "You are a helpful assistant who is summarizing student opinions on if phones should be banned from school."
        response = client.chat.completions.create(
            model="gpt-4o-mini-2024-07-18", 
            messages=[
                {"role": "system", "content": system_message},
                {"role": "user", "content": user_prompt}
            ],
            response_format=sentiment_schema,
            max_tokens=150  # Adjust as needed
        )
        return json.loads(response.choices[0].message.content)
    except Exception as e:
        return {"error": f"An error occurred: {str(e)}"}

def store_result(result, filename="sentiment_results.json"):
    try:
        # Read existing results
        with open(filename, 'r') as f:
            results = json.load(f)
    except FileNotFoundError:
        # If file doesn't exist, start with an empty list
        results = []
    
    # Append new result
    results.append(result)
    
    # Write updated results back to file
    with open(filename, 'w') as f:
        json.dump(results, f, indent=2)
    print(f"Result appended to {filename}")


# Test the function
if __name__ == "__main__":
    test_prompt = "I think phones should be banned from school since they are distracting from my learning."
    result = gpt4_call(test_prompt)
    
    # Store the result in the file
    store_result(result)

Result appended to sentiment_results.json


Deleting Contents in the JSON file

In [66]:
import json
import os

def clear_sentiment_results(filename="sentiment_results.json"):
    try:
        # Open the file in write mode, which will clear its contents
        with open(filename, 'w') as f:
            # Write an empty list to the file
            json.dump([], f)
        print(f"All objects removed from {filename}. The file now contains an empty list.")
    except Exception as e:
        print(f"An error occurred while clearing the file: {str(e)}")

# Call the function to clear the file
clear_sentiment_results()

All objects removed from sentiment_results.json. The file now contains an empty list.


New Code with tracking of reasonings and updating prompt

In [67]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI

# Load environment variables
load_dotenv()

# Access the API key
api_key = os.getenv('OPENAI_API_KEY')

# Create an OpenAI client
client = OpenAI(api_key=api_key)

# Initialize the previous_results dictionary
previous_results = {}

# Define the sentiment schema
sentiment_schema = {
    "type": "json_schema",
    "json_schema": {
        "name": "sentiment_response",
        "strict": True,
        "schema": {
            "type": "object",
            "properties": {
                "sentiment": {
                    "type": "boolean",
                    "description": "True if in favor of the proposition, false if against"
                },
                "reason": {
                    "type": "string",
                    "description": "Very brief core argument in sentiment analysis"
                }
            },
            "required": ["sentiment", "reason"],
            "additionalProperties": False
        }
    }
}

def gpt4_call(question, opinion):
    try:
        # Create a string of previous results and their frequencies
        previous_results_str = ", ".join([f"{reason} ({count} times)" for reason, count in previous_results.items()])
        
        system_message = f"""You are a helpful assistant analyzing opinions on the following question: '{question}'.
        Previous core arguments: {previous_results_str if previous_results else 'None'}.
        Analyze the given opinion and provide a boolean sentiment (true if in favor, false if against) and a very brief core argument.
        If the opinion's core argument is similar to a previous one, use that exact wording."""
        
        response = client.chat.completions.create(
            model="gpt-4o-mini-2024-07-18", 
            messages=[
                {"role": "system", "content": system_message},
                {"role": "user", "content": opinion}
            ],
            response_format=sentiment_schema,
            max_tokens=150  # Adjust as needed
        )
        result = json.loads(response.choices[0].message.content)
        
        # Update the previous_results dictionary
        reason = result.get('reason')
        if reason:
            previous_results[reason] = previous_results.get(reason, 0) + 1
        
        return result
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None, None

def store_result(result, filename="sentiment_results.json"):
    try:
        # Read existing results
        with open(filename, 'r') as f:
            results = json.load(f)
    except FileNotFoundError:
        # If file doesn't exist, start with an empty list
        results = []
    
    # Append new result
    results.append(result)
    
    # Write updated results back to file
    with open(filename, 'w') as f:
        json.dump(results, f, indent=2)

# Main execution
if __name__ == "__main__":
    question = "Should phones be banned from schools?"
    opinions = [
        "I think phones should be banned from school since they are distracting from students learning.",
        "Phones are necessary for emergency situations, so they shouldn't be banned.",
        "Banning phones would eliminate cheating during exams.",
        "Students need phones to stay connected with their parents.",
        "Phones can be educational tools when used properly in the classroom."
    ]

    for opinion in opinions:
        store_result(gpt4_call(question, opinion))

    print("\nCore Argument Frequencies:")
    for reason, count in previous_results.items():
        print(f"{reason}: {count}")


Core Argument Frequencies:
phones are distracting from students learning: 2
phones are necessary for emergency situations: 2
Banning phones would eliminate cheating during exams.: 1
